In [1]:
# Results for A100

In [2]:
import torch

In [3]:
def _fused_add(x, y, alpha):
    x += alpha * y


fused_add = torch.compile(_fused_add)


def _fused_sub(x, y, alpha):
    x -= alpha * y


fused_sub = torch.compile(_fused_sub)


def _fused_mul_add(p, z, beta):
    p *= beta
    p += z


fused_mul_add = torch.compile(_fused_mul_add)

In [4]:
n = 100_000_000

In [5]:
x = torch.randn(n, device="cuda")
y = torch.randn(n, device="cuda")
alpha = torch.randn(1, device="cuda")

In [6]:
def timeit(fun, warmup_iters=20, iters=100, repetitions=5):
    times = []
    for _ in range(repetitions):
        start = torch.cuda.Event(enable_timing=True)
        end = torch.cuda.Event(enable_timing=True)

        for _ in range(warmup_iters):
            fun()

        start.record()
        for _ in range(iters):
            fun()
        end.record()
        torch.cuda.synchronize()

        times.append(start.elapsed_time(end) / iters)

    return min(times)

In [7]:
timeit(lambda: _fused_add(x, y, alpha))

1.173411865234375

In [8]:
timeit(lambda: fused_add(x, y, alpha))

0.6717030334472657